# Import the requierments

In [1]:
# !pip install -qU transformers==4.51.3 datasets==3.2.0
# !pip install -qU openai==1.61.0 wandb
# !pip install -qU json-repair==0.29.1 numpy chardet
# !pip install -qU vllm==0.7.2 optimum==1.24.0

In [1]:
from google.colab import userdata
import wandb

wandb.login(key=userdata.get('wandb'))
hf_token = userdata.get('huggingface')
!huggingface-cli login --token {hf_token}

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mo7amed3twan (mo7amed3twan-kafr-el-sheikh-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Qwen 0.5` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Qwen 0.5`


In [2]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Prepare The Data For Finetuning

In [92]:
data_dir = "/content/Data"

device = "cuda"

base_model_id = "Qwen/Qwen2.5-0.5B"

# YoussefHosni/Qwen2.5-0.5b-arabic-2B-toekn-finetuned-model

# def parse_json(text):
#     try:
#         return json_repair.loads(text)
#     except:
#         return None

### Load The Mode

In [93]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = None
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

#### **Show The Model Architecture**

In [94]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

### **Apply My Schema On The Model Chat Templet**

In [80]:
qa_message = [{"role": "user", "content": "ما هي المظاهر الاقتصادية فى دولة مصر ؟"}]

In [81]:
text = tokenizer.apply_chat_template(
    qa_message,
    tokenize=False,
    add_generation_prompt=True
)

In [82]:
text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nما هي المظاهر الاقتصادية فى دولة مصر ؟<|im_end|>\n<|im_start|>assistant\n'

### **Start encode the prompt to IDS**

In [83]:
model_inputs = tokenizer([text], return_tensors="pt").to(device)

In [84]:
model_inputs

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 124009, 128420,  53479,  92381,
         125387, 136251, 128393,  44330, 125138, 128748,   8803,    253, 151645,
            198, 151644,  77091,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

### **Generate The Response In IDS**

In [85]:
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=2048,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [86]:
generated_ids

tensor([[151644,   8948,    198,  ...,  73274, 135022, 139013]],
       device='cuda:0')

### **Here I Pick up just The Response of the model in also IDS**

In [87]:
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [88]:
generated_ids

[tensor([124130, 126198,  73274,  ...,  73274, 135022, 139013], device='cuda:0')]

### **Last Step that I Decode The IDS Into Text**

In [90]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [91]:
print(response)

كل ما يخص الاقتصاد المصري هو تراجع معدلات النمو في عام 2007 و2008 وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 2007 2,5% و2008 2,7% وانه لم يتجاوز معدلات النمو في عام 200

# Now Let Us Finetuning Our Model

In [ ]:
sft_data_path = join(data_dir, "dataset", "DATA.jsonl")
sft_data_path

'/kaggle/working/youtube-resources/llm-finetuning/dataset/DATA.jsonl'

In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
# ```
   "QAtrain": {
        "file_name": "/kaggle/input/hadiths/llamafactory-finetune-data-v2/train.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    },
    "QAval": {
        "file_name": "/kaggle/input/hadiths/llamafactory-finetune-data-v2/val.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TQDM"] = "1"


In [ ]:
%%writefile /kaggle/working/LLaMA-Factory/examples/train_lora/QA.yaml

### model
model_name_or_path: Qwen/Qwen2.5-0.5B
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: QAtrain
eval_dataset: QAval
template: qwen
cutoff_len: 4096
max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 32

### output
resume_from_checkpoint: "/kaggle/working/Models_Ouput2/last-checkpoint"
output_dir: "/kaggle/working/Models_Ouput2"
logging_steps: 50
save_steps: 500
plot_loss: true
overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true # full
ddp_timeout: 180000000

### eval
val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: epoch
eval_steps: 500

report_to: wandb
run_name: Qwennn

push_to_hub: true
export_hub_model_id: "Atwan/QWEN_Arabic_RAG"
hub_private_repo: true
hub_strategy: checkpoint

Writing /kaggle/working/LLaMA-Factory/examples/train_lora/QA.yaml


In [ ]:
%cd /kaggle/working

/kaggle/working


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /kaggle/working/LLaMA-Factory/examples/train_lora/QA.yaml


2025-05-16 17:45:07.130929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747417507.155022     384 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747417507.162282     384 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[INFO|2025-05-16 17:45:14] llamafactory.hparams.parser:401 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.bfloat16
[INFO|tokenization_utils_base.py:2060] 2025-05-16 17:45:14,982 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-0.6B/snapshots/6130ef31402718485ca4d80a6234f70d9a4cf362/vocab.json
[INFO|tokenization_utils_base.py:2060] 2025-05-1

# Evaluate After Fine tuning

In [ ]:
finetuned_model_id = "/content/resources/llm-finetuning/Models/checkpoint-520/"
# finetuned_model_id = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_id,
    device_map="auto",
    torch_dtype = None
)

tokenizer = AutoTokenizer.from_pretrained(finetuned_model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
def generate_resp(qa_message):

    device = "cuda" if torch.cuda.is_available() else "cpu"

    text = tokenizer.apply_chat_template(
        qa_message,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [5]:
question_text = [{"role": "user","content": "ما هي المظاهر الاقتصادية فى دولة مصر ؟"}]

In [9]:
response = generate_resp(question_text)
response

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

تُهيمن القطاعات الخدمية (قناة السويس، السياحة، الاتصالات) على الهيكل الاقتصادي المصري، بينما تشهد القطاعات الإنتاجية (التصنيع، الزراعة، الطاقة) نموًا قويًا مدعومًا بمشروعات البنية التحتية الكبرى والاستثمارات الحكومية.

